## **함수목록** - 모듈화 결과물

### 0. 데이터 불러오기

In [ ]:
import pickle
import gzip

# 제공받은 csv의 랜덤한 10% 추출한 pickle file
with gzip.open(r'/content/drive/MyDrive/log-data/df_final_0.pickle','rb') as f:
     d = pickle.load(f)

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sklearn.feature_extraction.text import CountVectorizer
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

### 1. 전처리

In [ ]:
# 최종 전처리 함수
d["id"] = d["game_id"].astype(str) +"_"+ d["gamer_id"]
d2 = d.groupby("id").filter(lambda x : len(x)>10) # 11개이상의 로그를 가진 유저들의 데이터만 사용
d2["inDate"] = pd.to_datetime(d2["inDate"])
d2 = d2.sort_values(by="inDate", ascending=True)
d2["url"] = d2["url"].apply(lambda x : x.split("/")[2]).replace("gameInfo","gameinfo")
d2["action"] = d2["method"]+"_"+ d2["url"] #action 정의
df = d2.drop(["game_id","gamer_id","url","method","tableAndColumn"],axis= 1) #id,action,inDate 제외 columns drop

### 2. 기준일 설정

In [ ]:
# 기준일 설정
min = df['inDate'].min() # Timestamp('2021-12-28 09:34:38.909000')
min_date = min.to_pydatetime().date() # datetime.date(2021, 12, 28)
max = df['inDate'].max() # Timestamp('2022-01-11 23:59:59.995000')
max_date = max.to_pydatetime().date() # datetime.date(2022, 1, 11)

# 기준일 -n일 , +m일 설정
n = 5
m = 3

# 기준일 - 4일마다 나눔
target_days = (max_date - min_date).days // 4 # 12.28 ~ 01.11 까지 4일마다 기준일
target_days

# 기준일 목록 - timestmap 추출
target_list = [(max - timedelta(days=3*i)).replace(hour=10, minute=0, second=0, microsecond=0) for i in range(1, target_days +1)]
target_list.sort()
target_list

[Timestamp('2022-01-02 10:00:00'),
 Timestamp('2022-01-05 10:00:00'),
 Timestamp('2022-01-08 10:00:00')]

### 3. 임베딩

In [ ]:
# df에 라벨링 함수
def assign_churn_df(df, i):
  n = 5
  m = 3
  df = df[(df['inDate'] >= target_list[i] - timedelta(days=n)) & (df['inDate'] <= target_list[i] + timedelta(days=m))]
  n_df = df[(df['inDate'] < target_list[i]) & (df['inDate'] >= target_list[i] - timedelta(days=n))]
  m_df = df[(df['inDate'] > target_list[i]) & (df['inDate'] <= target_list[i] + timedelta(days=m))]
  churn_users = set(df.id) - (set(n_df.id) & set(m_df.id))
  df = df.assign(churn=0) #churn column
  df.loc[df.id.isin(churn_users), 'churn'] = 1
  return df

# data의 유저별 action을 나타내는 preprocessing 함수
def preprocessing_df(df):
  groupby_ID = df.groupby("id")
  ID_action_list =[]
  for key, value in groupby_ID: # 각 id당 inDate와 action을 나타냄
    ID_action_dictionary = {}
    ID_action_dictionary["id"] = key
    ID_action_dictionary["action"] = value["action"].to_list() #action의 value값을 리스트로 적용
    ID_action_list.append(ID_action_dictionary)
  return pd.DataFrame(ID_action_list)

# 유저별 actions를 빈도수로 count vectorize 하는 함수 
def user_vectorize(df):
  vector = CountVectorizer()
  corpus = df["action"].apply(lambda x : " ".join(x))
  count_df = pd.DataFrame(vector.fit_transform(corpus).toarray(),
                          columns=vector.get_feature_names_out(),
                          index = df['id']
                          )
  return count_df

### 4. 라벨링

In [ ]:
# 유저별 합친 데이터에 churn 라벨링 함수
# vector화 후 target_df[i] 넣으면 사용 가능
def assign_churn_train(df,id):
  df = df.assign(churn=0) #churn column
  df.loc[df.index.isin(id), 'churn'] = 1
  return df

# 라벨링된 df의 churn user id 리스트 저장 함수
def list_churn_id(df):
  churn_users = list(df['id'].loc[df['churn'] == 1])
  return churn_users

### 5. 함수 적용한 데이터

In [ ]:
# df with churn
# 0 = 기준일 2022-01-02 10:00:00 : target_list[0]
# 1 = 기준일 2022-01-05 10:00:00 : target_list[1]
# 2 = 기준일 2022-01-08 10:00:00 : target_list[2]
df1 = assign_churn_df(df,0)
df2 = assign_churn_df(df,1)
df3 = assign_churn_df(df,2)  

# (OPTIONAL) 1000개 미만 빈도수의 action은 drop
counts = df['action'].value_counts() # value counts
df1 = df1[~df1['action'].isin(counts[counts < 1000].index)]
df2 = df2[~df2['action'].isin(counts[counts < 1000].index)]
df3 = df3[~df3['action'].isin(counts[counts < 1000].index)]


# 유저별 action 합친 후 데이터프레임 without churn
count_df1 = preprocessing_df(df1)
count_df2 = preprocessing_df(df2)
count_df3 = preprocessing_df(df3) 

# tf-idf 데이터프레임 without churn 
vector_count_df1 = user_vectorize(count_df1)
vector_count_df2 = user_vectorize(count_df2)
vector_count_df3 = user_vectorize(count_df3)

# tf-idf 데이터프레임에 churn 라벨링
train_df1 = assign_churn_train(vector_count_df1,list_churn_id(df1))
train_df2 = assign_churn_train(vector_count_df2,list_churn_id(df2))
train_df3 = assign_churn_train(vector_count_df3,list_churn_id(df3))


### 6. 모델링 전 데이터 x, y split

In [ ]:
# 기준일1 기준 데이터 -  2021-12-28 10:00:00.278 ~ 2022-01-05 09:59:59.798
y_train_1 = train_df1.loc[:,'churn']
X_train_1 = train_df1.drop('churn', axis=1)

# 기준일2 기준 데이터 -  2021-12-31 10:00:00.258 ~ 2022-01-08 09:59:59.760
y_train_2 = train_df2.loc[:,'churn']
X_train_2 = train_df2.drop('churn', axis=1)

# 기준일3 기준 데이터 -  2022-01-03 10:00:00.154 ~ 2022-01-11 09:59:59.841
y_train_3 = train_df3.loc[:,'churn']
X_train_3 = train_df3.drop('churn', axis=1)

7. XGBoost 모델 - 기준일 1+2 학습 후 기준일3 예측

### 7. 기본 XGBoost Model

In [ ]:
xgb = XGBClassifier()

# 기준일 1 학습
xgb.fit(X_train_1,y_train_1)

# 기준일 2 예측
y_pred_1 = xgb.predict(X_train_2)
roc_score = roc_auc_score(y_train_2, y_pred_1)
print('ROC AUC 값 : {0:.4f}'.format(roc_score))

ROC AUC 값 : 0.7493


### 8. randomsearchCV 튜닝 - estimator로 예측경우

In [ ]:
from sklearn.model_selection import KFold, RandomizedSearchCV

model = XGBClassifier()

xgb_parameters = {
 'min_child_weight': [0.5, 1, 3, 5, 8],
 'gamma': [0.2, 0.5, 1, 2],
 'subsample': [0.4, 0.6, 0.8, 1.0],
 'colsample_bytree': [0.2, 0.4, 0.6, 0.8],
 'max_depth': [4, 6, 8, 10],
 'objective': ['binary:logistic'],
}

cv = KFold(n_splits=6)

rsv = RandomizedSearchCV(model, xgb_parameters, cv=cv, scoring='roc_auc', n_jobs=6, verbose=10)
rsv.fit(X_train_1.values,y_train_1.values)
estimator = rsv.best_estimator_

print('final params', rsv.best_params_)
print('best score', rsv.best_score_)

Fitting 6 folds for each of 10 candidates, totalling 60 fits
final params {'subsample': 0.6, 'objective': 'binary:logistic', 'min_child_weight': 0.5, 'max_depth': 8, 'gamma': 0.5, 'colsample_bytree': 0.4}
best score 0.7932100278438324


In [ ]:
# 파라미터 튜닝후 기준일 1 학습
estimator.fit(X_train_1,y_train_1)

# 파라미터 튜닝후 기준일 2 예측
y_pred_1 = estimator.predict(X_train_2)
roc_score = roc_auc_score(y_train_2, y_pred_1)
print('ROC AUC 값 : {0:.4f}'.format(roc_score))

ROC AUC 값 : 0.7557


### 9. SMOTE 오버샘플링

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_1_smote,y_train_1_smote = smote.fit_resample(X_train_1,y_train_1)
X_train_2_smote,y_train_2_smote = smote.fit_resample(X_train_2,y_train_2)
X_train_3_smote,y_train_3_smote = smote.fit_resample(X_train_3,y_train_3)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train_1.shape, y_train_1.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_1_smote.shape, y_train_1_smote.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_1_smote).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (72662, 23) (72662,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (86596, 23) (86596,)
SMOTE 적용 후 레이블 값 분포: 
 0    43298
1    43298
Name: churn, dtype: int64


In [ ]:
# SMOTE 오버 샘플링 / 파라미터 튜닝 후 기준일 1 학습
estimator.fit(X_train_1_smote,y_train_1_smote)

# SMOTE 오버 샘플링 / 파라미터 튜닝 후  기준일 2 예측
y_pred_1 = estimator.predict(X_train_2)
roc_score = roc_auc_score(y_train_2, y_pred_1)
print('ROC AUC 값 : {0:.4f}'.format(roc_score))

ROC AUC 값 : 0.7598
